<a href="https://colab.research.google.com/github/bhavanapavuluri/reimagined-octo-funicular/blob/master/pavuluribhavana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Name:** Bhavana Pavuluri

**Internship project topic:** Automate extraction of handwritten text from an image.

**Name of the Organization:** TCS iON

**Name of the Industry Mentor:** Anamika Chatterjee

**Name of the Institute:** St.PETER'S Engineering College

Installation Module: You have to install tesseract-ocr engine and pytesseract which is a wrapper of tesseract-ocr engine

In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract


In this cell you will import all the required modules or libraries

In [2]:
import pytesseract
import shutil
import os
from PIL import Image

Input cell: In this we will create an upload button by which you can easily upload images from your PC.Also it retrieves the image from which the text is to be extracted

In [ ]:
from google.colab import files

uploaded = files.upload()

for filename in uploaded.keys():
    print(uploaded[filename])

Image which is used to test this project

![alt text](https://i.pinimg.com/236x/3e/4e/31/3e4e31eb6d94daf2901894268022c048--typography-instagram.jpg)

You can use or download IAM handwriting database images at [link text](http://www.fki.inf.unibe.ch/databases/iam-handwriting-database)


Output cell: In this we have done image processing and text extraction

In [16]:
import cv2 
import pytesseract
import numpy as np

# read image
im = cv2.imread(filename)

# preprocessing
# adjust contrast and brightness
def contrast_brightness(img):
    contrast_img = cv2.addWeighted(img, 2.5, np.zeros(img.shape, img.dtype), 0, 20)
    cv2.imwrite(r'img_contrast.png',img)
    return contrast_img

# gray scale
def gray(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cv2.imwrite(r"img_gray.png",img)
    return img  

# detect and correct the text skew
def dcskew(img):
    img=cv2.bitwise_not(img)
    coordinates = np.column_stack(np.where(img > 0))
    ang=cv2.minAreaRect(coordinates)[-1]
    if ang<-45:
        ang=-(90+ang)
    else:
        ang=-ang
    height, width = img.shape[:2]
    center = (width / 2, height / 2)    
    rotationMatrix = cv2.getRotationMatrix2D(center, ang, 1.0)
    rotated_img = cv2.warpAffine(img, rotationMatrix, (width, height), borderMode = cv2.BORDER_REFLECT)
    cv2.imwrite("img_rotated.png",rotated_img)
    return img

# blur
def blur(img) :
    img_blur = cv2.GaussianBlur(img,(5,5),0)
    cv2.imwrite(r"img_blur.png",img)    
    return img_blur

# detect edges
def edges(img):
    edge_img = cv2.Canny(img,100,200)
    cv2.imwrite("edge_img.png",img)
    return edge_img

# threshold
def threshold(img):
    #pixels with value below 100 are turned black (0) and those with higher value are turned white (255)
    img = cv2.threshold(img, 225, 255, cv2.THRESH_BINARY_INV)[1]  
    mask = img.copy()
    mask = cv2.erode(mask, None, iterations=5)
    mask = cv2.dilate(mask, None, iterations=5)
    output = cv2.bitwise_and(img, img, mask=mask)
    cv2.imwrite(r"img_threshold.png",img)
    return img    

# Finding contours 
im_contrast_brightness = contrast_brightness(im)
im_gray = gray(im_contrast_brightness)
im_dcskew = dcskew(im_gray)
im_blur = blur(im_dcskew)
im_edges = edges(im_blur)
im_thresh = threshold(im_edges)

contours, _ = cv2.findContours(im_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 
# text detection
def contours_text(orig, img, contours):
    for cnt in contours: 
        x, y, w, h = cv2.boundingRect(cnt) 

        # Drawing a rectangle on copied image 
        rect = cv2.rectangle(orig, (x, y), (x + w, y + h), (0, 255, 255), 2) 
      
        # Cropping the text block for giving input to OCR 
        cropped = orig[y:y + h, x:x + w] 
        cv2.imwrite(r"croppped_image.png",img)

im_contours = contours_text(im,im,im)
cv2.imwrite(filename,im)
# Apply OCR on the cropped image 
text = pytesseract.image_to_string(Image.open(filename))
print(text)

YOURE
ACTUALLY
WOWDERFUL
